In [ ]:
#Author: Oğuzhan Öztürk to get contact: oguzhanozturk0@outlook.com

!pip install bertopic

In [ ]:
!pip install bertopic[flair]
!pip install bertopic[gensim]
!pip install bertopic[spacy]
!pip install bertopic[use]

In [ ]:
from bertopic import BERTopic

In [ ]:
!pip install --upgrade bertopic

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk import word_tokenize,sent_tokenize 

stop_words=nltk.corpus.stopwords.words('english')
new_stopwords = ['movie','movies','film','love','good','like','best','great','ring','shawshank','films','one','ive','watch','thing','amazing','greatest','well','ever','']
stop_words.extend(new_stopwords)

def clean_text(review):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(review)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text

In [ ]:
docs = pd.read_csv('YOUR CSV FILE PATH', on_bad_lines='skip')['review']

In [ ]:
docs = docs.astype(str)
docs=docs.apply(clean_text)

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))


In [ ]:
!pip install numpy --upgrade


In [ ]:
!pip install gensim==3.8.0
!pip install numpy==1.19.0


In [ ]:
import gensim
from gensim.corpora import Dictionary

dictionary=Dictionary(docs)
corpus=[dictionary.doc2bow(review) for review in docs]
topic_model = BERTopic(corpus, nr_topics=12, id2word=dictionary, verbose=True, n_gram_range=(1, 1),min_topic_size=10,low_memory=False)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

In [ ]:
topic_model.get_topic(0)

In [ ]:
topic_model.reduce_topics(docs, nr_topics=12)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model = BERTopic(verbose=True, n_gram_range=(2, 2),calculate_probabilities=True)
topics, probs = topic_model.fit_transform(docs)

In [ ]:
topic_model.reduce_topics(docs, nr_topics=12)

In [ ]:
topic_model.visualize_distribution(probs[0])

In [ ]:
topic_model.visualize_topics()

In [ ]:
calculate_probabilities=True
fig = topic_model.visualize_distribution(topic_model.probabilities_[0])
fig.write_html("path/to/file.html")


In [ ]:
from bertopic import BERTopic
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

topic_model = BERTopic(verbose=True, n_gram_range=(1, 3))
topics, _ = topic_model.fit_transform(docs)

# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()